In [1]:
import pandas as pd

from df.keiba_data_frame import KeibaDataFrame
from result.result_frame import ResultFrame

# Train Data 読み込み
kdf = KeibaDataFrame.from_datas(["./../data/train/2004_ranking_without_g.csv"])

# Train Data 整備
# ※ 1: G1, 2: G2:, 3: G3, 4: 新馬, 5: 未勝利, 6: 未出馬, 0: それ以外
kdf.filter_datas([0, 4, 5, 6])
#kdf.setting_datas()
kdf.setting_types()

# Result Data 読み込み
rdf = ResultFrame.from_data("./../data/train/2004_result.csv")

# Result Data 整備
rdf.setting_datas()
rdf.setting_types()


In [2]:
import numpy as np
from tensorflow import keras

new_model = keras.models.load_model("./../model/ranking_model_without_g")

preds = []

i = 0
for v in kdf.groups():
    obs = v.reindex(range(20), fill_value=-1).values.reshape(1, 1, 20, 50).tolist()
    q_values = new_model.predict(obs)
    action = np.argmax(q_values[0])
    pred_df = pd.DataFrame(
        {
            "track_id": v["track_id"],
            "date_num": v["date_num"],
            "round": v["round"],
            "horse_number": v["number"],
            "action": action,
        }
    )
    preds.append(pred_df)
    i += 1
    if i > 100:
        break

preds_df = pd.concat(preds)

1/1 [==============================] - 0s 31ms/step


In [3]:
kounyu = []
atari = []
hazure = []
kati = []
make = []
mikounyu = []

df = preds_df.sort_values(
    ["date_num", "track_id", "round"], ascending=[True, True, True]
)

i = 0
for _, v in df.groupby(["date_num", "track_id", "round"], sort=False):
    check_key = v.head(1)
    check_target = v.head(6)
    result_target = rdf.df.loc[
        (rdf.df["track_id"] == int(check_key["track_id"]))
        & (rdf.df["date_num"] == int(check_key["date_num"]))
        & (rdf.df["round"] == int(check_key["round"]))
    ]
    action = int(check_key["action"])
    horse_1 = check_target.head(1)
    horse_2 = check_target.head(2).tail(1)
    horse_3 = check_target.head(3).tail(1)
    horse_4 = check_target.head(4).tail(1)
    horse_5 = check_target.head(5).tail(1)

    kounyu = True
    if action == 0:
        make.append(1000)
        ok_horse = result_target[
            (result_target["number"] == int(horse_1["horse_number"]))
        ]
    elif action == 1:
        make.append(1000)
        ok_horse = result_target[
            (result_target["number"] == int(horse_2["horse_number"]))
        ]
    elif action == 2:
        make.append(1000)
        ok_horse = result_target[
            (result_target["number"] == int(horse_3["horse_number"]))
        ]
    elif action == 3:
        make.append(1000)
        ok_horse = result_target[
            (result_target["number"] == int(horse_4["horse_number"]))
        ]
    elif action == 4:
        make.append(1000)
        ok_horse = result_target[
            (result_target["number"] == int(horse_5["horse_number"]))
        ]
    else:
        ok_horse = []
        kounyu = False

    hit = False
    if len(ok_horse) != 0:
        kati.append(float(ok_horse["odds"]) * 1000)
        hit = True

    if hit:
        atari.append(1)
    elif kounyu:
        hazure.append(1)
    else:
        mikounyu.append(1)

print(f"未購入回数: {sum(mikounyu)}")
print(f"当たり馬券: {sum(atari)}")
print(f"はずれ馬券: {sum(hazure)}")
print(f"的中率: {(sum(atari) / sum(hazure))}")
print(f"収入: {sum(kati)}")
print(f"支出: {sum(make)}")
print(f"回収率: {(sum(kati) / sum(make))}")

未購入回数: 1
当たり馬券: 8
はずれ馬券: 92
的中率: 0.08695652173913043
収入: 80700.0
支出: 100000
回収率: 0.807
